In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
%cd /kaggle/working/
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

/kaggle/working
fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.


/kaggle/working/chatGLM-6B-QLoRA


In [ ]:
#cat /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0812-v1/checkpoint-40/adapter_config.json

In [29]:
%cd chatGLM-6B-QLoRA
!git pull --all --force 
!pip install -r requirements.txt

/kaggle/working/chatGLM-6B-QLoRA
Fetching origin
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), 4.24 KiB | 434.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   d947d02..3013da0  main       -> origin/main
Updating d947d02..3013da0
Fast-forward
 ppo_chatglm2.py | 33 +++++++++++++++++++++++++++------
 1 file changed, 27 insertions(+), 6 deletions(-)


# 

In [ ]:
cat /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0811-v3/config.json

In [ ]:
!git pull --all --force
!CUDA_VISIBLE_DEVICES=0 python  rm_trl.py \
--model_name 'THUDM/chatglm2-6b' \
--resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/reward_model_0809_v1/checkpoint-50 \
--num_train_epochs 2 \
--gradient_accumulation_steps 1 \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size  4 \
--max_length 512 \
--output_dir ./reward_model_0810_v2 \
--train_subset 80 \
--eval_subset 20 \
--local_rank 0  \
--bf16 False

# 上面的warning There were missing keys in the checkpoint model loaded: 'transformer.embedding.word_embeddings.weight', 'transformer.rotary_pos_emb.inv_freq', 'transformer.encoder.layers.0.input_layernorm.weight', ....
# 是 trainingArguments中设置 load_best_model_at_end = True 后出现的
# 说明 trainer的确在最后把最好的那个adapters的参数做了load（只不过load时strict=True要求严格匹配了）
# 这说明保存在output_dir中的pytorch_model.bin和vhead.bin参数都是最佳的 我们来验证下

验证成功  跑的是py中的test_load_best()


# deepspeed 单机多卡从头训练 reward model

In [ ]:
!git pull --all --force 
!deepspeed --include localhost:0,1  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v2 \
  --num_train_samples 200 \
  --num_eval_samples 100 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  2  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json


--deepspeed ds_zero2_config.json

# 单机多卡deepspeed 从ckpt继续训练 

In [ ]:
!git pull --all --force 
!deepspeed --include localhost:0,1  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v3 \
  --resume_from_checkpoint output-rm-1k-0811-v2 \
  --num_train_samples 200 \
  --num_eval_samples 100 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  2  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

In [ ]:
ls -lrt /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0811-v3

# 单卡训练 reward model

In [ ]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v1 \
  --num_train_samples 20 \
  --num_eval_samples 20 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  10  \
  --save_total_limit 2 \
  --load_in_4bit True 


In [ ]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rm_3.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0812-v1 \
  --num_train_samples 20 \
  --num_eval_samples 20 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 400 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  4  \
  --save_total_limit 2 \
  --load_in_4bit True 

In [ ]:
rm -rf output-rm-1k-0812-v1

参考 https://github.com/hiyouga/ChatGLM-Efficient-Tuning/blob/main/src/glmtuner/tuner/core/adapter.py#L83
实现 resume——from --checkpoint

参考

In [ ]:
https://huggingface.co/docs/trl/models#trl.AutoModelForCausalLMWithValueHead
    https://github.com/yongzhuo/chatglm-maths/blob/main/chatglm_maths/t10_lora_trl_train_ppo.py#L175
        https://huggingface.co/docs/trl/v0.5.0/en/customization#use-the-cuda-cache-optimizer
            https://huggingface.co/docs/trl/trainer#trl.RewardTrainer

# python 【rm_3.py】 单卡训练 从头开始

In [ ]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rm_3.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0812-v1 \
  --num_train_samples 20 \
  --num_eval_samples 20 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 400 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  4  \
  --save_total_limit 2 \
  --load_in_4bit True 

# deepspeed train 【rm_3.py】恢复训练 
#  batach_size 2  gradient accumulation steps 4

In [ ]:
!git pull --all --force 
!deepspeed --include localhost:0,1   rm_3.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0812-v2 \
  --resume_from_checkpoint output-rm-1k-0812-v1/checkpoint-60 \
  --num_train_samples 500 \
  --num_eval_samples 200 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2  \
  --gradient_accumulation_steps 4 \
  --learning_rate  2e-5 \
  --num_train_epochs  4  \
  --save_total_limit 2 \
  --load_in_4bit True 

In [4]:
ls output-rm-1k-0812-v2

README.md
adapter_config.json
adapter_model.bin
checkpoint-110/
checkpoint-120/
events.out.tfevents.1691831244.e711a0a4361e.539.0
events.out.tfevents.1691831976.e711a0a4361e.593.0
events.out.tfevents.1691833306.a62342d81a09.196.0
events.out.tfevents.1691833749.a62342d81a09.298.0
events.out.tfevents.1691834459.a62342d81a09.394.0
value_head.bin


In [6]:
ls /kaggle/working/chatGLM-6B-QLoRA/data/rlhf-reward-single-round-trans_chinese/

test-00000-of-00001-8ecd46436fadcf7f.parquet
train-00000-of-00001-789dc5dece0f1fc1.parquet


# PPO training 

In [51]:
!git pull --all --force
!CUDA_VISIBLE_DEVICES=0,1 python ppo_chatglm2.py \
    --base_model_name THUDM/chatglm2-6b \
    --merged_sft_model_path THUDM/chatglm2-6b \
    --sft_model_lora_path ./output-rm-1k-0812-v2 \
    --reward_model_lora_path ./output-rm-1k-0812-v2 \
    --adafactor False \
    --save_freq 10 \
    --output_max_length 256 \
    --batch_size 1 \
    --gradient_accumulation_steps 1 \
    --batched_gen True \
    --ppo_epochs 4 \
    --seed 0 \
    --learning_rate 1e-5 \
    --early_stopping True \
    --output_dir ppo_0813_v1 \
    --log_with tensorboard

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 687 bytes | 343.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   6f58a8a..3fab8b9  main       -> origin/main
Updating 6f58a8a..3fab8b9
Fast-forward
 ppo_chatglm2.py | 1 +
 1 file changed, 1 insertion(+)
[2023-08-13 17:50:18,068] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorfl

ls: cannot access './output-rm-1k-0812-v2/adapter_model.bin': No such file or directory
